## Scala Algorithms

By: Alex Comerford (alexanderjcomerford@gmail.com)

In this notebook we will be going over some simple algorithms implemented scala along with examples and utility functions

This notebook is based off of the works of
* http://scalacookbook.com/
* https://github.com/ashawley/algs/
* https://github.com/vkostyukov/scalacaster
* https://github.com/kimxogus/algorithms-scala/

## Binary search

This is a fundamental algorithm which searches through a sequence of sorted integers by partitioning the entire sequence into 2, doing a single comparison, then repeating until 1 element remains. This is awesome because at most this search does *O* (log n) comparisons

In [203]:
import scala.util.{Try, Random}

// utility function to generate a random list of ordered numbers
def sampleRemove[A](items:List[A], remove:Int) = {
    def collect(vect: Vector[A], sampleSize: Int, acc : List[A]) : Vector[A] = {
        if (sampleSize == 0) {
            vect
        } else if (vect.length == 0) {
            vect
        } else {
            val index = Random.nextInt(vect.size)
            collect( vect.patch(index, Nil, 1), sampleSize - 1, vect(index) :: acc)
        }
    }
    collect(items toVector, remove, Nil)
}

// num to search, all nums
def search(k: Int, a: Seq[Int]) = {
    @annotation.tailrec
    def search2(lo: Int, mid: Int, hi: Int): Int = {
      Try(a(mid)).toOption match {
        case Some(x) if lo > hi => -1
        case Some(x) if x == k => mid
        case Some(x) if x > k => search2(lo, (mid - lo) / 2, mid - 1)
        case Some(x) if x < k => search2(mid + 1, mid + 1 + (hi - mid) / 2, hi)
        case None => -1
      }
    }
    search2(0, a.length / 2, a.length - 1)
}

import scala.util.{Try, Random}
sampleRemove: [A](items: List[A], remove: Int)Vector[A]
search: (k: Int, a: Seq[Int])Int


## Search a list of 1000 elements

In the next cell we will use these two functions to search a list

In [204]:
val randList = sampleRemove(items = 1 to 10000 toList, 
                            remove = 9000)
val randItem = randList(Random.nextInt(1000))

println(s"Searching for ${randItem}")
val index = search(randItem, randList)
println(s"Item located at index $index, item value = ${randList(index)}")

Searching for 2471
Item located at index 253, item value = 2471


null

## Sorting!

In this next cell we will put two sorting algorithms

The Bubble sorting algorithm is one of the most simplest sorting algorithms out there and is extremely easy to implement. At a whopping effecency of О(n^2) this algorithm is slooooow given a large array compared to other Time complexities. However it is extremely easy to read and can be easily implemented recursively

The MergeSorting sorting algorithm is one of my personal favorites because of it's clevar use of recusion. Even though its space requirement isn't the most efficent (Θ(n) compared to other more efficent Θ(1)), it's design and implementation accross languages is consistently simple and elegant.

In [205]:
def bubbleSort(a: List[Int]): List[Int] = {
    def bubble(a: List[Int]): List[Int] = {
        a match {
            case Nil => Nil
            case a :: Nil => a :: Nil           // when the end of a list is reached, return
            case a :: b :: aa =>                // comparison!
                if (b < a) b :: bubble(a :: aa) // reverse position defined in case
                else a :: bubble(b :: aa)       // keep position defined in case
        }
    }
    a match {
        case Nil => Nil
        case a :: aa => bubble(a :: bubbleSort(aa))
    }
}

def mergeSort(a: List[Int]): List[Int] = {
    def merge(a: List[Int], b: List[Int]): List[Int] = {
        (a, b) match {
            case (Nil, b) => b                       // no left entry to compare, return 
            case (a, Nil) => a                       // no left entry to compare, return
            case (a :: aa, b :: bb) =>               // comparison!!! we are sorting left to right
                if (a < b) a :: merge(aa, b :: bb)   // order stays just like case, merge right
                else b :: merge(a :: aa, bb)         // switch order, merge left
        }
    }
    // match some base cases to end early
    a match {
        case Nil => Nil
        case x :: Nil => a
        case a => {
            val half = a.length / 2
            val (a1, a2) = a.splitAt(half) // hooray pattern matching!
            merge(sort(a1), sort(a2))
        }
    }
}

bubbleSort: (a: List[Int])List[Int]
mergeSort: (a: List[Int])List[Int]


## Sorting 1000 elements

In this next cell we will sort 1000 randomly shuffled elements using these two functions and compare performance!

SPOILER ALERT!!!!
Bubble sort is slower

In [206]:
import scala.math.pow
def time[R](block: => R): R = {
    val t0: Double = System.nanoTime()
    val result = block    // call-by-name
    val t1: Double = System.nanoTime()
    println("Elapsed time: " + (t1 - t0) * pow(10,-9) + "s")
    result
}

{ 
    time { mergeSort { Random.shuffle { 1 until 1000 toList } } }
}
{
    time { bubbleSort { Random.shuffle { 1 until 1000 toList } } } 
}

Elapsed time: 0.0041125960000000005s
Elapsed time: 0.220723041s


null

## Roman numeral conversion

This is a little and cute algorithm to convert base10 integers into roman numerals. Roman numerals are an interesting number system because they are an additive and subtractive number system based on certain axiomatic symbols put in different combinations to represent a numeric value.

In this next cell we will convert integers to roman numerals by use of greatest element subtraction where elements are searched in a descending lookup table. This table could be arbitrarily complex depending how detailed you want your subtractive or additive notation to be.

In [207]:
def intToRoman(x: Int): String = {
    val digits = List(1000 ->  "M", 900 -> "CM", 500 ->  "D", 
                       400 -> "CD", 100 ->  "C",  90 -> "XC", 
                        50 ->  "L",  40 -> "XL",  10 ->  "X", 
                         9 -> "IX",   5 ->  "V",   4 -> "IV", 
                         1 ->  "I")

    def loop(l: List[(Int, String)], y: Int): String =
      if (y == 0) ""
      else l.head match {
        case (v, s) if (v <= y) => s + loop(l, y - v)
        case _ => loop(l.tail, y)
      }

    loop(digits, x)
}

intToRoman: (x: Int)String


In [208]:
val randList = sampleRemove(items = 1 to 1000 toList, 
                            remove = 990)

randList.map{_.toInt} foreach(e => println(intToRoman(e)))

XXXVII
CCCLVII
CCCLXXXIII
DXXXV
DLXXXV
DCLXXIX
DCCCXLIV
CMVII
CMXXXVII
CMXL


null

## Flatten

This is fairly straight forward algorithm to flatten an potentially multi nested array. One interesting use case of flatten in the python numpy package is to flatten 2D pixel arrays into a single vector (see https://www.safaribooksonline.com/library/view/programming-computer-vision/9781449341916/ch01.html)

In [209]:
def flatten(e: Any): List[Any] = {
    e match {
        case Nil => Nil
        case head :: rest => flatten(head) ++ flatten(rest)
        case v => List(v)
    }
}

val nested1 = List(List(List(1,2), List(3,4)), List(List(5,6)))
val nested2 = List(List(List(1,2, 3,4)), List(List(5,6)))
println(flatten(nested1))
println(flatten(nested2))

List(1, 2, 3, 4, 5, 6)
List(1, 2, 3, 4, 5, 6)


null

## GCD and LCM

These are some pretty fundamental number theory algorithms greatest common divisor and least common multiple

In [210]:
def gcd(x: Int, y: Int): Int = 
    if (y == 0) x else gcd (y, x % y)

def lcm(x: Int, y: Int): Int = 
    math.abs(x * y) / gcd(x, y)

gcd: (x: Int, y: Int)Int
lcm: (x: Int, y: Int)Int


In [211]:
println(gcd(5010,486))
println(gcd(5010,483))
println(lcm(5010,486))
println(lcm(5010,483))

6
3
405810
806610


null

## Palindrome

There are much easier ways to do this but iterating indices from beggining to end and end to beggining comparing characters is a pretty good algorithm


In [212]:
def isPalindrome(s: String): Boolean = {
    def loop(i: Int, j: Int): Boolean =
      if (i >= j) true
      else if (s.charAt(i) == s.charAt(j)) loop(i + 1, j - 1)
      else false
    loop(0, s.length - 1)
}

isPalindrome: (s: String)Boolean


In [213]:
println(isPalindrome("tacocat"))
println(isPalindrome("potato"))

true
false


null